15分でできる日本語Word2Vec https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

In [59]:
import json

In [60]:
path = 'data/recipe_chefgohan.json'
df = json.load(open(path, 'r', encoding='utf-8'))
# print(df)
text = ''
for _,r in df['recipes'].items():
    # print(r['title'])
    start = r['title']+'を作ります。'
    end = r['title']+'が完成しました。'
    instructions = ''.join(r['instructions']).replace('\n','')
    text += start + instructions + end + '\n'
print(text)


さつま芋のもちもちチーズ羊羹を作ります。牛乳と生クリームを合わせ、棒寒天を加えて煮溶かす。下準備したさつまいもとクリームチーズを合わせてよく混ぜてSTEP1をこしながら加える。さらに小麦粉とラム酒を加え中火で練り込み、全体がポッテリしてきたら荒熱をとって流し缶に流して冷蔵庫で固める。クリームバターの材料を合わせ混ぜておく。固まった羊羹を適量切り出してクリームバターを少々かけ、ミントを飾る。さつま芋のもちもちチーズ羊羹が完成しました。
つけ麺パスタを作ります。調味料を全て合わせ、卵黄を加え良く混ぜておく。ベーコンをバターで炒め胡椒を軽くふっておく。器に(1)を適量注ぎ、(2)を適量と粉チーズをお好みの量入れ、粉パセリを振る。パスタをアルデンテ（少し芯が残る程度）に茹で冷水で冷まし、オリーブ油を絡ませておく。(4)を皿に盛り(3)を添える。つけ麺パスタが完成しました。
アロエプリンを作ります。アロエを水気をよくとってミキサーにかけておく。牛乳と生クリームを合わせ、溶けた寒天をちぎって加えて火にかけて煮溶し、西京みそ、グラニュー糖、トレハロースを加える。STEP2にSTEP1を加えて一煮立ちしたら火を止めてゼラチンを加えてザルでこしてブランデーを加える。シロップ用の水と砂糖を合わせて火にかけ、砂糖が溶けたらシロップ用のアロエを加え、一煮立ちしたらアクをとり、火を止めてラム酒を加えて冷ます。黒みつ用のアロエを水気をとって黒みつ適量で和える。器にシロップを適量入れ、荒熱をとったプリン液を適量注ぎ冷蔵庫で固め、黒みつ用のアロエを適量盛り、セルフィーユを飾る。アロエプリンが完成しました。
カキの雲丹みそ　紅葉焼きを作ります。白みそから下の材料を鍋に入れて弱火でよく練りながら火を入れ玉みそを作る。玉みそに蒸した雲丹をきざんで、ゆでた野菜のせん切りと共に混ぜる。カキは、さっと湯通しして氷水に落し、水気を切る。カキを殻に戻して、雲丹みそを一面にかけて焼く。カキの雲丹みそ　紅葉焼きが完成しました。
春の茶碗蒸し〜明日葉仕立て〜を作ります。明日葉にかつお出汁200ccを加えて、ミキサーにかけてクッキングペーパーで裏ごす。（裏ごしした明日葉は銀あん用に適量とっておく。）STEP1にかつお出汁を足して500ccにして調味料を各小さじ1加え、溶きほぐした卵を加えて裏ごす。飛び魚、しいたけを器に入

In [61]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

さつま芋
もち
もち
チーズ
羊羹
作る


sentences=None, 
corpus_file=None, 
vector_size=100, 
alpha=0.025, 
window=5, 
min_count=5, 
max_vocab_size=None, 
sample=0.001, 
seed=1, 
workers=3, 
min_alpha=0.0001, 
sg=0, 
hs=0, 
negative=5, 
ns_exponent=0.75, 
cbow_mean=1, 
hashfxn=<built-in function hash>, 
epochs=5, 
null_word=0, 
trim_rule=None, 
sorted_vocab=1, 
batch_words=10000, 
compute_loss=False, 
callbacks=(), 
comment=None, 
max_final_vocab=None)

In [62]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
# print(help(word2vec.Word2Vec()))
model = word2vec.Word2Vec(word_list, vector_size=300,min_count=5,window=5,epochs=100)

In [63]:
print(model.__dict__['wv']['肉'])

[-0.08262171 -0.1950612   2.0241246  -1.314554    1.140346   -2.352511
  1.4122416  -1.6616114  -0.38343796 -0.3236279  -2.0578582   2.2909255
 -1.224376   -0.46360794 -0.15338361 -0.8538006   0.08093034 -0.4198177
 -0.16244721 -0.10451014  0.9283389   1.3456618   0.20923622 -0.6516728
 -0.09295174 -1.1452142  -0.80092734 -0.1921925   0.08474257 -1.342977
 -1.2262284   0.63582134  0.43607268  0.60904366  1.0650473   0.06674059
  0.36338148  1.6976408   1.0468969  -1.3845801   0.07998727 -1.0097668
 -0.6287484   2.2280931   1.0389831   0.61673635  1.1187772  -0.1149742
  0.59936357 -0.990211    0.9585306   0.07122104 -0.15204449 -1.9328884
 -0.23921008  1.5587798   1.2716453  -0.47744757  1.227892   -0.5146825
  0.9968644   1.0414803   0.54659045  0.71657264  0.14881982  0.52025634
 -0.91072977  1.4312006  -1.045708   -0.6143261  -0.785685   -1.3371326
  0.17824718 -1.0462574   0.58522767  0.9876617   0.19895506  0.8513664
  0.99624455  0.29267225 -0.76223016  0.3207967  -1.2269939   0.

In [64]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['焼く']) 
for item in ret:
    print(item[0], item[1])

焼ける 0.5209134817123413
キツネ 0.48692622780799866
焼き上げる 0.45134061574935913
トースター 0.44982409477233887
返す 0.43373772501945496
押す 0.3948192000389099
面 0.3915705680847168
裏返し 0.38488999009132385
焼き 0.37494421005249023
ぬる 0.37487852573394775


In [65]:
results = model.wv.most_similar(positive=['肉','たまねぎ'], topn=5)
for result in results:
    print(result[0])

挽き肉
ロース
ピーマン
きゅうり
セロリ


In [74]:
results = model.wv.most_similar(positive=['鶏'], topn=5)
for result in results:
    print(result[0])

牛
トリ
手羽
もも肉
つき


In [66]:
wv = model.__dict__['wv']['たまねぎ']
print(wv)

[ 0.6068811   0.19817695 -0.24645525  0.2876079  -0.32943946  0.2839342
  0.90727687  0.5889373  -0.29716778 -0.49811295  0.15777056  1.3804897
  0.17715205  0.37563765  0.31871325  0.40244183 -1.1936266   1.1756837
 -0.5137457  -0.82072747 -0.10339563 -0.23937134  0.82369065  0.27581432
 -0.5709943   0.37434703  0.61407137 -0.15899223 -0.3850924   0.12718435
  0.5323845   0.29070106  0.25081643 -0.32768917  0.7294462  -0.18151042
 -1.4377127  -0.08156659  0.6195818   1.3055445  -0.26186758  1.4039478
  0.46454844 -1.1099118  -0.07708918  0.4131894   0.9871842   1.6997494
 -0.18065116  0.20333736 -0.50675005  0.04655349 -0.82887506 -0.12697831
  0.12300938  0.24111935  0.6566292   0.94967073  0.45067316 -0.06265547
 -0.69650155 -1.0214674   0.90696937 -0.56539834  0.61800784 -1.2431629
  0.15429583 -0.8122002  -0.04850793  1.2030662   0.18188205 -0.21623176
  0.48013625  1.0794832   1.1500332  -0.41206935  0.201342    0.09306528
 -0.23370078 -0.4565317  -0.40759903 -0.47548336  0.03014